<a id='toc'></a>
* [Часть 1. Особенности LU-разложения](#lu)
    * [LU-разложение с выбором главного элемента (по столбцу)](#plu)
        * [Задание 1.2 (1 балл)](#task_12)
        * [Задание 1.3 (1 балл)](#task_13)
* [Часть 2. Решение СЛАУ с положительно определённой матрицей](#solve)
* [Часть 3. Разреженные матрицы](#sparse)
* [Часть 4. Итерационные методы](#iterative)

Не забываем загрузить библиотеки:

In [180]:
# Библиотека для работы с матрицами
import numpy as np 

# Алгоритмы линейной алгебры
import scipy.linalg as sla

# Библиотека для работы с разреженными матрицами
import scipy.sparse as sps

# Алгоритмы линейной алгебры для разреженных матриц
import scipy.sparse.linalg as spla

# Графическая библиотека
import matplotlib.pyplot as plt

# Библиотека для измерения времени
import timeit

# Позволяет отрисовывать графики и изображения прямо в ноутбуке, а не в отдельном окне. Жизненно важная вещь!
%matplotlib inline

<a id='lu'></a>
# Часть 1. Особенности LU-разложения [[toc](#toc)]

С точки зрения математики матричные разложения являются точными: произведение сомножителей всегда равняется исходной матрицы $A$. К сожалению, на практике этом часто мешает вычислительная погрешность. 

Для $LU$ разложения l2-норма ошибки ошибки $||\delta A|| = ||A - LU||$ удовлетворяет следующей оценке:

$$||\delta A|| \leqslant ||L|| \cdot ||U|| \cdot O(\varepsilon_{machine})$$

А нормы $L$ и $U$ могут быть совсем нехорошими.

<a id='lu_task11'></a>
### Задание 1.1 (1 балл)  [[toc](#toc)]
**Рассмотрим следующее LU-разложение:**

$$\begin{pmatrix}
10^{-20} & 1\\
1 & 1
\end{pmatrix} = \begin{pmatrix}
1 & 0\\
10^{20} & 1
\end{pmatrix}\cdot\begin{pmatrix}
10^{-20} & 1\\
0 & 1 - 10^{20}
\end{pmatrix}$$

Перемножьте полученные матрицы $L$ и $U$. А теперь перемножьте такие же матрицы, только после всех единиц поставьте десятичные точки. Изменился ли ответ? Как вам кажется, почему?

In [5]:
L = np.array([[1,    0], 
              [1e20, 1]])
U = np.array([[1e-20, 1], 
              [0,     1 - 1e20]])
A = np.array([[1e-20, 1], 
              [1,     1]])
print(A)
print(np.dot(L, U))

[[  1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00]]
[[  1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00   0.00000000e+00]]


Отметим, что в реальных вычислениях матричные элементы почти наверняка с самого начала будут числами с плавающей точкой (а не целыми).

Теперь проверьте, что будет, если вычислить QR-разложение исходной матрицы и перемножить матрицы $Q$ и $R$.

In [67]:
A = np.array([[1e-20, 1], 
              [1,     1]])
N = A.shape[0]
qr = QR()
Q, R = qr(A)
print('Matrices:')
print('A:\n', A)
print('Q:\n', Q)
print('R:\n', R)
E_rec = np.dot(Q, Q.T)
A_rec = np.dot(Q, R)
print('QR:\n', A_rec)

print('\nErrors:')
print('||E - QQ^T|| = {}'.format(np.sum((np.eye(N) - E_rec)**2)))
print('||A - QR|| = {}'.format(np.sum((A - A_rec)**2)))

Matrices:
A:
 [[  1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00]]
Q:
 [[  1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00  -1.00000000e-20]]
R:
 [[ 1.  1.]
 [ 0.  1.]]
QR:
 [[  1.00000000e-20   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00]]

Errors:
||E - QQ^T|| = 0.0
||A - QR|| = 0.0


In [50]:
class QR:
    def __init__(self):
        pass
    def __call__(self, A):
        R = np.array(A)
        assert R.ndim == 2
        assert R.shape[0] == R.shape[1]
        N = R.shape[0]
        T = np.eye(N)
        for i in range(N - 1):
            for j in reversed(range(i + 1, N)):
                x = R[i, i]
                y = R[j, i]
                l = np.sqrt(x**2 + y**2)
                c =  x / l
                s = -y / l
                ri = c * R[i, i:] - s * R[j, i:]
                rj = s * R[i, i:] + c * R[j, i:]
                R[i, i:] = ri
                R[j, i:] = rj
                ti = c * T[i, :] - s * T[j, :]
                tj = s * T[i, :] + c * T[j, :]
                T[i, :] = ti
                T[j, :] = tj
        Q = T.T.copy()
        if R[N - 1, N - 1] < 0:
            R[N - 1, N - 1] *= -1
            Q[:, N - 1] *= -1
        return Q, R

In [51]:
np.random.seed(1)
N = 5
A = np.random.rand(N, N)
qr = QR()
Q, R = qr(A)
#print('Q:\n', Q)
#print('R:\n', R)
print(R[N - 1, N - 1])
E_rec = np.dot(Q, Q.T)
print('||E - QQ^T|| = {}'.format(np.sum((np.eye(N) - E_rec)**2)))
A_rec = np.dot(Q, R)
print('||A - QR|| = {}'.format(np.sum((A - A_rec)**2)))


0.599986097161
||E - QQ^T|| = 6.401791135143172e-31
||A - QR|| = 3.29332139882927e-31


<a id='plu'></a>
### Выход: LU-разложение с выбором главного элемента (по столбцу)  [[toc](#toc)]
Каждый раз ищем максимум в столбце и переставляем соответствующую строку наверх.

$$\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{blue}{b_{ii}} & \color{blue}{b_{i,i+1}} & \dots & \color{blue}{b_{in}} \\
 & & b_{i+1,i} & b_{i+1,i+1} & \dots & b_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & \vdots & \vdots & & \vdots\\
\end{pmatrix}\longrightarrow
\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & b_{i+1,i} & b_{i+1,i+1} & \dots & b_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & \color{blue}{b_{ii}} & \color{blue}{b_{i,i+1}} & \dots & \color{blue}{b_{in}} \\
 & & \vdots & \vdots & & \vdots\\
\end{pmatrix}\longrightarrow$$
$$\longrightarrow\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & 0 & b'_{i+1,i+1} & \dots & b'_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & 0 & b'_{i,i+1} & \dots & b'_{in} \\
 & & \vdots & \vdots & & \vdots
\end{pmatrix}$$

Надо сказать, что примерно так вы все и решали системы на первом курсе университета! Именно наибольший, а не первый ненулевой элемент столбца берётся потому, что чем больше число - тем меньшие погрешности потенциально вносит деление на него.

Что при этом происходит? Перестановка строк матрицы равносильна умножению её слева на матрицу соответствующей перестановки. Таким образом, мы получаем равенство

$$L_nP_nL_{n-1}P_{n-1}\ldots L_2P_2L_1P_1 A = U\qquad\qquad(1)$$

где $L_1,\ldots,L_n$ - некоторые нижнетреугольные матрицы.

**Вопрос:** Ну, и где здесь матрица $L$?!

**Ответ:** Введём новые матрицы

\begin{align*}
L'_n &= L_n\\
L'_{n-1} &= P_nL_{n-1}P_{n}\\
L'_{n-2} &= P_nP_{n-1}L_{n-2}P_{n-1}P_{n}\\
&\ldots\\
L'_1 &= P_nP_{n-1}\ldots P_2L_1P_2\ldots P_{n-1}P_n
\end{align*}

**Упражнение.** Матрицы $L'_i$ тоже нижнетреугольные!

Тогда левая часть (1) перепишется в виде

$$\underbrace{L'_nL'_{n-1}\ldots L'_1}_{:=L^{-1}}\underbrace{P_nP_{n-1}\ldots P_1}_{:=P^{-1}}\cdot A$$

\begin{gather}
L = P_n P_{n-1}\dots P_{2}P_{1}\cdot P_1L^{-1}_1P_2L^{-1}_2P_3L^{-1}_3P_4\dots L^{-1}_{n-2}P_{n-1} L^{-1}_{n-1}P_n L_{n}\\
P = P_1 P_2 \dots P_{n-1}P_n
\end{gather}

**Итог:** разложение вида
$$A = PLU$$
где $P$ - матрица перестановки.

Функция `scipy.linalg.lu` в Питоне находит именно такое разложение!

Все элементы $L$ не превосходят $1$, так что $||L|| \leqslant 1$. При этом
$$||\Delta A|| \leqslant ||A||\cdot O(\rho \varepsilon_{machine}),$$
где
$$\rho = \frac{\max_{i,j}|u_{ij}|}{\max_{i,j}|a_{ij}|}$$

Но что, если это отношение велико?

<a id='task_12'></a>
### Задание 1.2 (1 балл) [[toc](#toc)]
**Сгенерируйте матрицу $500\times500$, имеющую вид**
$$\begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0 & 1\\
-1 & 1 & 0 &  &  & 0 & 1\\
-1 & -1 & 1 & 0  &  & 0 & 1\\
\vdots & & \ddots & \ddots  & \ddots & \vdots & \vdots \\
-1 & -1 & -1 & \ddots & 1 & 0 & 1\\
-1 & -1 & -1 &  & -1 & 1 & 1\\
-1 & -1 & -1 & \cdots & -1 & -1 & 1
\end{pmatrix}$$

Например, сгенерировать сначала нулевую матрицу нужного размера, а потом заполнить её клетки правильными числами.

Найдите её PLU-разложение и QR-разложение. Убедитесь, что $P = E$. Вычислите $||A - LU||_2$ и $||A - QR||_2$. Чему равен фактор роста матрицы $A$?

**[TODO: Что такое фактор роста?]**

In [195]:
N = 500
A = np.zeros((N, N))
A[np.arange(N), np.arange(N)] = 1
A[:, N - 1] = 1
for i in range(N - 1):
    A[i + 1:, i] = -1
P, L, U = sla.lu(A)
Q, R = sla.qr(A)

assert np.all(P == np.eye(N))
assert np.allclose(np.dot(Q, Q.T), np.eye(N))

A_plu = np.dot(np.dot(P, L), U)
A_qr = np.dot(Q, R)
A_eigvals = sla.eigvals(A)
eig_max, eig_min = np.max(np.abs(A_eigvals)), np.min(np.abs(A_eigvals))
print('A eig max = {}, eig min = {}'.format(eig_max, eig_min))
print('eig max / eig min = {}'.format(eig_max / eig_min))
print('\nErrors:')
print('||A - PLU|| = {}'.format(np.sum((A - A_plu)**2)))
print('||A - QR|| = {}'.format(np.sum((A - A_qr)**2)))

A eig max = 93.93348742729968, eig min = 1.499797752056208
eig max / eig min = 62.63076958111038

Errors:
||A - PLU|| = 6.415845591476438e+152
||A - QR|| = 2.4129274267093647e-24


In [175]:
class PLU:
    def __init__(self):
        pass
    def _print(self, msg):
        if self.verbose:
            print(msg)
    def __call__(self, A, verbose=False, debug=False):
        A = A.copy()
        assert A.ndim == 2
        N = A.shape[0]
        L_cols = []
        Ps = []
        self.verbose = verbose
        if debug:
            self.P_matrices = {}
            self.L_matrices = {}
        for i in range(N):
            j = i + np.argmax(A[i:, i])
            self._print('A[{}, {}] = {}, A[{}, {}] = {}'.format(i, i, A[i, i], i, j, A[j, i]))
            Ps.append((i, j))
            if debug:
                P = np.eye(N)
                P[i, i] = 0; P[i, j] = 1
                P[j, j] = 0; P[j, i] = 1
                self.P_matrices[i] = P
                
            row = A[i, :].copy()
            A[i, :] = A[j, :]
            A[j, :] = row
            
            a = A[i, i]
            assert a != 0
            self._print('Maximum element at column i = {} is {}'.format(i, a))
            L_col         = np.zeros(N)
            L_col[i]      = 1.0 / a
            self._print('column i={} before reduction: {}'.format(i, A[i:, i]))
            L_col[i + 1:] = -A[i + 1:, i] / a
            A[i + 1:, i:] += L_col[i + 1:, None] * A[i, i:][None, :]
            A[i, i:]      /= a
            self._print('column i={} after reduction: {}'.format(i, A[i:, i]))
            L_cols.append(L_col)
            if debug:
                L = np.eye(N)
                L[:, i] = L_col
                self.L_matrices[i] = L
        self._print('matrix U:\n{}'.format(A))
        
        # Восстановление матриц преобразования
        P_l = np.eye(N)
        P_r = np.eye(N)
        L   = np.eye(N)
        
        if debug:
            for i in reversed(range(N)):
                P_i = self.P_matrices[i]
                L_i = self.L_matrices[i]
                a = 1.0 / L_i[i, i]
                L_i[i, i] = a
                L_i[i + 1:, i] *= -a
                L = np.dot(L_i, L)
                L = np.dot(P_i, L)
                P_l = np.dot(P_i, P_l)
                P_r = np.dot(P_r, P_i)
        else:
            for i in reversed(range(N)):
                v, u = Ps[i]
                assert i == v

                # Finding L_i * L_{i-1}'
                L_i = np.eye(N)
                #T = L_i.copy()
                L_col = L_cols[i]
                #T[:, i] = L_col
                a = 1.0 / L_col[i]
                L_col[i] = a
                L_col[i + 1:] *= -a
                L_i[:, i] = L_col
                L = np.dot(L_i, L)

                # Finding P_i * (L_i * L_{i-1}')
                row = L[v].copy()
                L[v] = L[u]
                L[u] = row

                # Finding P_1 P_2 P_3 ... P_{n-1} P_n
                row = P_l[v].copy()
                P_l[v] = P_l[u]
                P_l[u] = row

                # Finding P_n P_{n-1} ... P_3 P_2 P_1
                col = P_r[:, v].copy()
                P_r[:, v] = P_r[:, u]
                P_r[:, u] = col
        L = np.dot(P_r, L)
        P = P_l
        return P, L, A

In [ ]:
np.random.seed(18)
N = 25
A = np.random.rand(N, N)
plu = PLU()
P, L, U = plu(A, debug=False)
print_matrices = False
A_rec = np.dot(np.dot(P, L), U)

if print_matrices:
    print('Matrices:')
    print('A:\n', A)
    print('P:\n', P)
    print('L:\n', L)
    print('U:\n', U)
    print('PLU:\n', A_rec)
    
print('\nErrors:')
print('||A - PLU|| = {}'.format(np.sum((A - A_rec)**2)))

К счастью, на практике так редко бывает (чёрт его знает почему). Тем не менее, QR-разложение всё-таки лучше. Теоретическая оценка для ошибки QR-разложения имеет вид

$$||A - QR||_2 \leqslant ||A||_2\cdot O(\varepsilon_{machine})$$

<a id='task_13'></a>
### Задание 1.3 (1 балл) [[toc](#toc)]
**Рассмотрим *матрицу Паскаля* $S_n = \left(C_{i + j}^i\right)$ ($i,j = 0,\ldots,n-1$).**

**Каково её LU-разложение? Выведите формулы для матриц L и U и приведите краткое обоснование прямо в ноутбуке. Не пользуйтесь функцией `scipy.linalg.lu`, чтобы его "угадать": матрица P будет отлична от единичной, и вы получите не то, что хотели.**

**Каков её определитель?**

----
*Ваш ответ*

Напишите функцию `my_pascal(n)`, генерирующую матрицу Паскаля размера $n\times n$.

Найдите норму разности $||A - PLU||_2$. Не такая уж и большая, правда?

In [ ]:
###
# Your function here

###

A = my_pascal(30)

# Find ||A - PLU||_2 here

Теперь попросим компьютер вычислить определитель матрицы Паскаля $30\times30$ и решить простенькую систему уравнений:

In [ ]:
print(sla.det(A))

# Try to solve a linear system
x = np.ones(30)
b = A.dot(x)
x1 = sla.solve(A, b)
print(sla.norm(x1 - x))

Так себе ошибка. Теперь попробуем сделать это с помощью QR-разложения. Станет ли лучше?

In [ ]:
Q, R = sla.qr(A)
x2 = sla.solve_triangular(R, Q.T.dot(b))
print sla.norm(x2 - x)

Объясните полученные неприятные результаты.

----
*Ваш ответ*

<a id='solve'></a>
# Часть 2. Решение СЛАУ с положительно определённой матрицей [[toc](#toc)]

**Задание 2.1. Какие же методы использовать? (3 балла)**

Реализуйте несколько алгоритмов решения СЛАУ $Ax = b$, где $A = A^T$, $A \geqslant 0$ с матричной правой частью $b$.

1. **Наивный способ**: $x = A^{-1}b$;

2. **Стандартный способ**: с помощью процедуры `solve` модуля `scipy.linalg`;

3. **Разложение Холецкого**: с помощью разложения Холецкого для матрицы $A$ и последующего решения двух СЛАУ с треугольными матрицами;

4. **Разложение Холецкого с процедурами scipy**: с помощью разложения Холецкого для матрицы $A$ и специальных процедур из пакета `scipy.linalg` (`cho_factor`, `cho_solve`).

Для решения СЛАУ с треугольной матрицей можно воспользоваться функцией `solve_triangular` из пакета `scipy.linalg`.

In [ ]:
def naive_solve(A, b):
    # Your code here
    
# And so on and so forth

Проведите тестирование реализованных алгоритмов на небольшой СЛАУ на предмет совпадения ответов

Проведите эксперименты и выясните, как меняется время работы этих методов

- с ростом размера матрицы $A$ при фиксированном числе правых частей. Рассмотрите системы с 10, 100, 1000 правых частей;

- с ростом числа правых частей при фиксированном размере матрицы $A$ (например, $100\times100$).

Обязательно нарисуйте графики (время работы от размера). Какой метод оказывается более быстрым?

Для тестирования вам пригодятся случайные матрицы, сгенерированные с помощью функции `numpy.random.randn`. Но не забудьте, что в задаче речь идёт о *симметрических положительно определённых матрицах*. Так что подумайте, как из случайных матриц сделать симметрические положительно определённые.

Матрицы левых частей должны быть не менее $100\times100$: при меньших размерностях заметную роль могут играть факторы, не имеющие отношения к алгебре. Мы рекомендуем рассматривать системы с матрицами размера от 100 до 1000 и с числом правых частей от 10 до 10000. Приготовьтесь к тому, что эксперименты могут занять какое-то время.

**Задание 2.2. Пример: вычисление логарифма плотности многомерного нормального распределения (3 балла)**

Случайная величина $\vec{x}\in\mathbb{R}^D$ имеет многомерное нормальное распределение, если её плотность может быть представлена как
$$
p(\vec{x}) = \mathcal{N}(\vec{x}|\vec{\mu},\Sigma) = \frac{1}{\sqrt{2\pi}^D\sqrt{\det\Sigma}}\exp\left(-\frac{1}{2}(\vec{x}-\vec{\mu})^T\Sigma^{-1}(\vec{x}-\vec{\mu})\right)
$$
Здесь $\vec{\mu}\in\mathbb{R}^D -$ вектор мат. ожидания $\vec{x}$, а $\Sigma\in\mathbb{R}^{D{\times}D} -$ матрица ковариации.

С помощью матричных разложений реализуйте алгоритм вычисления логарифма нормальной плотности для набора векторов $X = \{\vec{x}_1,\dots,\vec{x}_N\}$ для заданных $\vec{\mu}$ и $\Sigma$.

In [ ]:
#Заготовка:
def my_multivariate_normal_logpdf(X, m, S):
    '''
    Ввод
    -----
    X: набор точек, numpy array размера N x D;
    m: вектор средних значений, numpy array длины D;
    S: ковариационная матрицы, numpy array размера D x D.
    
    Вывод
    ------
    res: результат вычислений, numpy array длины N.
    '''

Сгенерируйте выборку из нормального распределения со случайными параметрами для небольшого $D$ с помощью функции `scipy.stats.multivariate_normal.rvs` и сравните на этой выборке результат работы вашего алгоритма с результатом стандартной функции `scipy.stats.multivariate_normal.logpdf`

Замерьте время работы вашего алгоритма и функции `scipy.stats.multivariate_normal.logpdf` для различных значений $D$. Постарайтесь добиться, чтобы ваш алгоритм выигрывал по скорости у стандартной функции.

В задаче будут оцениваться:
- удалось ли вам обогнать библиотечную функцию;
- использованы ли вы матричные разложения (или просто обратили матрицу:))
- наличие дополнительных оптимизаций

<a id='sparse'></a>
# Часть 3. Разреженные матрицы [[toc](#toc)]

Матрица называется разреженной, если в ней мало ненулевых элементов.

Например, если в матрице $n\times n$ порядка $O(n)$ ненулевых элементов, она является разреженной.

Зачастую размерность разреженных матриц, возникающих в реальных задачах, так велика, что хранить её в памяти вместе с нулями - непозволительная роскошь. Есть несколько экономичных методов хранения:

- `Dictionary of Keys` (`DOK`) - словарь `(i,j):element`. 

    $\color{green}{\oplus}$ быстрое добавление элементов с произвольными индексами,
  
    $\color{red}{\ominus}$ любые другие операции будут производиться медленно.


- `List of Lists` (`LIL`) - матрица хранится построчно: в виде двух массивов `[l_1,...,l_s]` и `[v_1,...v_s]`, где `l_i` - список номеров столбцов, в которых в i-й строке находится ненулевой элемент, а `v_i` - список соответствующих значений. В целом, подходит для создания в высокой степени разреженной матрицы. Когда все элементы добавлены, лучше перевести в формат `CSR` или `CSC`.

    $\color{green}{\oplus}$ добавление за линейное время, 
    
    $\color{green}{\oplus}$ быстрый доступ к строкам матрицы, 
    
    $\color{red}{\ominus}$ может требовать слишком много памяти (для создания матриц повышенной плотности используйте `COO`).

    
- `Coordinate List` (`COO`) - хранятся тройки `(row, column, value)`  или три массива \texttt{rows,\ columns,\ values}. При этом тройка с одинаковым началом `(row, column)` может быть не одна; при преобразовании к другому типу значения `value` суммируются.

    $\color{green}{\oplus}$ быстрое добавление новых элементов,
    
    $\color{red}{\ominus}$ для всего остального лучше перевести в другой формат.
    
    
- `Compressed Sparse Row/Column storage` (`CSR/CSC`) - разберём на примере `CSR`. Хранятся три массива: `values`, `indptr` и `indices`. В массиве `values` хранятся все ненулевые элементы матрицы, упорядоченные лексикографически по паре (строка, столбец); `indptr[i]` - индекс начала `i`-й строки, `indices[indptr[i]:indptr[i+1]-1]` - номера столбцов соответствующих элементов.

    $\color{green}{\oplus}$ быстрое выполнение арифметических операций,
    
    $\color{green}{\oplus}$ быстрый доступ к строкам для `CSR` и к столбцам для `CSC`,
    
    $\color{red}{\ominus}$ очень медленный доступ к столбцам для `CSR` и к строкам для `CSC`,
    
    $\color{red}{\ominus}$ медленное добавление/удаление элементов.
  

Главный вывод - не надо один и тот же формат использовать для разных целей!

Вот здесь http://docs.scipy.org/doc/scipy/reference/sparse.html можно посмотреть, как эти возможности реализованы в библиотеке `scipy`.

Вот здесь https://www.cise.ufl.edu/research/sparse/matrices/index.html выложено много разреженных матриц из разнообразных приложений. Что особенно приятно, сайт предоставляет удобный клиент для скачивания, в котором доступен предпросмотр и данные о том, являются ли матрицы симметричными или положительно определёнными.

**Задание 3.0** Загрузите файл `sparse_matrix1.mtx`

In [ ]:
import scipy.io as sio

In [ ]:
A = sio.mmread(r'...\sparse_matrix1.mtx') # Please type right folder name! 

С помощью следующей функции можно посмотреть, как расположены ненулевые элементы матрицы:

In [ ]:
plt.spy(A, marker='.', markersize=0.4)

В каком из пяти форматов хранится матрица? Для ответа на этот вопрос воспользуйтесь функцию `type`.

Сколько в ней ненулевых элементов?

Посмотрим, сколько времени занимает преобразование между разными форматами.

In [5]:
import pandas as pd
import timeit

A_dok = A.todok()
A_lil = A.tolil()
A_csc = A.tocsc()
A_csr = A.tocsr()

conversion_times = pd.DataFrame(
    index=['COO', 'DOK', 'LIL', 'CSR', 'CSC'],
    columns=['COO', 'DOK', 'LIL', 'CSR', 'CSC'],
    data={
        'COO': [
            np.nan,
            timeit.timeit('A.todok()', 'from __main__ import A', number=100) / 100,
            timeit.timeit('A.tolil()', 'from __main__ import A', number=100) / 100,
            timeit.timeit('A.tocsr()', 'from __main__ import A', number=100) / 100,
            timeit.timeit('A.tocsc()', 'from __main__ import A', number=100) / 100,
               ],
        'DOK': [
            timeit.timeit('A_dok.tocoo()', 'from __main__ import A_dok', number=100) / 100,
            np.nan,
            timeit.timeit('A_dok.tolil()', 'from __main__ import A_dok', number=100) / 100,
            timeit.timeit('A_dok.tocsr()', 'from __main__ import A_dok', number=100) / 100,
            timeit.timeit('A_dok.tocsc()', 'from __main__ import A_dok', number=100) / 100,
               ],
        'LIL': [
            timeit.timeit('A_lil.tocoo()', 'from __main__ import A_lil', number=100) / 100,
            timeit.timeit('A_lil.todok()', 'from __main__ import A_lil', number=100) / 100,
            np.nan,
            timeit.timeit('A_lil.tocsr()', 'from __main__ import A_lil', number=100) / 100,
            timeit.timeit('A_lil.tocsc()', 'from __main__ import A_lil', number=100) / 100,
               ],
        'CSR': [
            timeit.timeit('A_csr.tocoo()', 'from __main__ import A_csr', number=100) / 100,
            timeit.timeit('A_csr.todok()', 'from __main__ import A_csr', number=100) / 100,
            timeit.timeit('A_csr.tolil()', 'from __main__ import A_csr', number=100) / 100,
            np.nan,
            timeit.timeit('A_csr.tocsc()', 'from __main__ import A_csr', number=100) / 100,
               ],
        'CSC': [
            timeit.timeit('A_csc.tocoo()', 'from __main__ import A_csc', number=100) / 100,
            timeit.timeit('A_csc.todok()', 'from __main__ import A_csc', number=100) / 100,
            timeit.timeit('A_csc.tolil()', 'from __main__ import A_csc', number=100) / 100,
            timeit.timeit('A_csc.tocsr()', 'from __main__ import A_csc', number=100) / 100,
            np.nan,
               ],
    }
    )

conversion_times.T

,COO,DOK,LIL,CSR,CSC
COO,NaN,0.064994,0.017649,0.000432,0.000536
DOK,0.104156,NaN,0.155717,0.103632,0.106420
LIL,0.014274,0.081863,NaN,0.014238,0.014916
CSR,0.000810,0.067233,0.013615,NaN,0.000494
CSC,0.000709,0.070901,0.015557,0.000509,NaN


Как вы можете убедиться, быстрее всего преобразования происходят между форматами `COO`, `CSR` и `CSC`, а хуже всего дела обстоят с форматом `DOK`: все преобразования из него занимают чудовищно много времени.

**Задание 3.1 (0,5 балла)** Почему преобразование из формата `LIL` в формат `CSR` занимает такую пропасть времени?

----
*Ваш ответ:*

**Задание 3.2 (1 балл)** Торговая сеть предоставила вам данные о покупках своих клиентов, представляющие собою список из нескольких сотен тысяч чеков (списков покупок). Для того, чтобы определить, какие товары чаще покупают вместе, вы решили построить матрицу, строки и столбцы которой соответствуют различным товарам (предположим, что число различных товаров тоже измеряется сотнями тысяч), а в клетке с "номером" $(g_1, g_2)$ стоит число

$\log_2{\frac{N\cdot c(g_1 \& g_2)}{c(g_1)c(g_2)}},$

где $c(g_i)$ --- количество чеков, содержащих товар $g_i$, $c(g_1 \& g_2)$ --- количество чеков, содержащих оба товара, $N$ --- общее число чеков. В каком формате вы будете создавать эту (очевидно разреженную) матрицу? Почему?

----
*Ваш ответ:*

Напишите максимально эффективный код, создающий эту матрицу:

In [ ]:
def CreateMatrix(receipts):
    # Your code here
    
    for receipt in receipts:
        # Your code here

**Задание 3.3 (1 балл)** В каком из форматов `LIL` и `COO` умножение на вектор происходит быстрее? Почему? Проведите эксперименты. Можете воспользоваться функцией `scipy.sparse.random` для создания случайных разреженных матриц.

Насколько быстрее с аналогичной задачей будут справляться форматы `CSC` и `CSR`?

----
*Ваш ответ:*

<a id='iterative'></a>
## Часть 4. Итерационные методы [[toc](#toc)]

В этом задании вам предлагается поработать с итеративными методами решения систем уравнений.

Соответствующие функции реализованы в пакете `scipy.sparse.linalg` (http://docs.scipy.org/doc/scipy-0.14.0/reference/sparse.linalg.html). Пожалуйста, читайте документацию перед их применением!

В этом задании вам предстоит поближе познакомиться с двумя итеративными методами:

1. **(L)GMRES** (мы настоятельно рекомендуем использовать оптимизированную функцию `scipy.sparse.linalg.lgmres`, даже если вам нужен обыкновенный **GMRES**)

2. **CG** (вызывается функцией `scipy.sparse.linalg.cg`)

*Замечания*:
1. Функции `scipy.sparse.linalg.lgmres` и `scipy.sparse.linalg.cs` устроены так, что могут решать уравнения только с векторной правой частью.
2. Внимательно ознакомьтесь с параметрами (у функции `scipy.sparse.linalg.lgmres` их очень много) и обратите внимание на формат вывода функций.
3. Вы можете захотеть выводить/сохранять что-нибудь после каждой итерации. Для этого существует параметр `callback`: это функция с сигнатурой `callback(xk)`, вызываемая на каждой итерации. Её аргумент `xk` - это текущее приближение $x_k$. Вот пример вызова функции `lgmres`, печатающей норму текущего приближения:

In [ ]:
x = spla.gmres(A, b, callback=lambda xk: print(sla.norm(xk))

Если вы захотите что-нибудь сохранять по ходу дела, логичнее использовать для этого класс. Ниже приводим пример класса, считающего число итераций и выводящего (если указан флаг `disp`) номер каждой итерации на печать, а также запоминающего все промежуточные приближения (не делайте так для больших систем! вам может не хватить памяти):

In [ ]:
class iterative_counter(object):
    def __init__(self, disp=True):
        self._disp = disp
        self.niter = 0
        self.all_x = [] # Please discard this if you solve large systems!!!
        
    def __call__(self, xk=None):
        self.niter += 1
        self.all_x.append(xk) # Please discard this if you solve large systems!!!
        if self._disp:
            print('iter %3i' % (self.niter))
            
my_counter = gmres_counter() # We need to create an instance of the class

x = spla.gmres(A, b, callback=my_counter)

print(my_counter.niter) # Will print total number of iterations

**Задание 4.1 (3 балла)** Довольно игр! Пора решать большие системы. Загрузите матрицу из файла `large_system.mtx` (она симметричная и положительно определённая) и сгенерируйте случайную правую часть. Решите систему с помощью функции `scipy.sparse.linalg.spsolve` (сильно оптимизированный "точный" решатель) и с помощью итеративных методов `LGMRES` и `CG`. Сравните скорость работы этих методов.

Постарайтесь обогнать функцию `spsolve`, применяя предобуславливание с помощью одной итерации метода Якоби или с помощью неполного LU-разложения. Для ILU постарайтесь подобрать оптимальные значения коэффициентов `fill_factor` и `drop_tol`.

*Замечание.* Если матрица-предобуславливатель $P$ не совсем уж тривиальная, не надо её обращать и умножать на исходную матрицу!

В каждом из итеративных методов можно включить предобуславливание с помощью параметра `M`. В качестве этого параметра нужно передать либо матрицу $P^{-1}$, либо **линейный оператор**, осуществляющий умножение вектора на $P^{-1}$. По понятным причинам второе гораздо правильнее. Делается это следующим образом. Например, если вы хотите ввести предобуславливание с конкретной матрицей $P$ для решения системы $Ax = b$:

In [ ]:
M = spla.LinearOperator(A.shape, lambda x: spla.spsolve(P, x))

x = spla.lgmres(A, b, M=M)

А вот как это работает для неполного LU-разложения:

In [ ]:
my_ILU = spla.spilu(A, '''Add your parameters here''')
M = spla.LinearOperator(A.shape, lambda x: my_ILU.solve(x))

x = spla.lgmres(A, b, M=M)

Обратите внимание, что `my_ILU` --- это не просто tuple из четырёх матриц (`spilu` делает разложение вида $P_1AP_2 = LU$, где $P_i$ --- матрицы перестановок). В самом деле, как вы уже, наверное, поняли, в мире больших размерностей иметь матрицу --- это зачастую бесполезное или даже вредное занятие. Гораздо ценнее уметь быстро решать систему с этой матрицей. Поэтому `my_ILU` --- это в первую очередь не разложение (впрочем, матрицы при желании тоже можно извлечь), а оптимизированный решатель `solve`.

In [ ]:
# Your solution here

## Часть 5. Матричные дифференцирования

**Задача 5.1 (1 балл)** Пусть $f$ --- функция на множестве квадратных матриц $n\times n$, а $g$ --- функция на множестве симметричных матриц $n\times n$, совпадающая с $f$ на своей области определения. Докажите, что

$$\frac{\partial g}{\partial X} = \frac{\partial f}{\partial X} + \left(\frac{\partial f}{\partial X}\right)^T -
\mathrm{diag}\left(\frac{\partial f}{\partial x_{11}}, \frac{\partial f}{\partial x_{22}},\ldots,
\frac{\partial f}{\partial x_{nn}}\right)$$

**Задача 5.2 (0.5 балла)** Найдите производную

$$\frac{\partial\mathrm{tr}\left(AX^2BX^{-T}\right)}{\partial X}$$

**Задача 5.3 (0.5 балла)** Найдите производную

$$\frac{\partial\ln{\det\left(X^TAX\right)}}{\partial X}$$

**Задача 5.4 (1 балл)** Допустим, что векторы $y_1,\ldots,y_m$ выбраны из многомерного нормального распределения с неизвестными вектором средних $m$ и ковариационной матрицей $\Sigma$. В этом задании вам нужно будет найти оценки максимального правдоподобия $\hat{m}$ и $\hat{\Sigma}$.

Напомним вкратце, что такое оценка максимального правдоподобия в случае непрерывного распределения. Пусть $p(x|\theta_1,\ldots,\theta_k)$ --- функция плотности распределения с неизвестными нам параметрами $\theta_1,\ldots,\theta_k$, а $y_1,\ldots,y_m$ --- выборка из этого распределения. \textit{Функцией правдоподобия} назовём произведение $L(y_1\ldots,y_m|\theta_1,\ldots,\theta_k) := \prod_{j=1}^kp(y_j|\theta_1,\ldots,\theta_k)$; грубо говоря, это произведение показывает, насколько правдоподобно появление данной выборки $y_1,\ldots,y_m$ при данных значениях параметров. В качестве оценки максимального правдоподобия выбирают те значения параметров, при которых функция правдоподобия достигает максимума. При этом как правило удобнее максимизировать не саму функцию правдоподобия, а *логарифмическую функцию правдоподобия* $l(y_1\ldots,y_m|\theta_1,\ldots,\theta_k) = \ln{L(y_1\ldots,y_m|\theta_1,\ldots,\theta_k)}$.

*Подсказка*. Постарайтесь превратить $\sum_i(x_i - m)^T\Sigma^{-1}(x_i - m)$ в функцию от матрицы $X$, столбцами которой являются векторы $x_i$.